In [ ]:
%pip install psycopg2
%pip install sqlalchemy

In [2]:
import psycopg2
from sqlalchemy import create_engine

# Function to create a new database
def create_database(db_name, username, password, host='localhost', port=5432):
    conn = psycopg2.connect(dbname="postgres", user=username, password=password, host=host, port=port)
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    cur = conn.cursor()
    cur.execute(f"CREATE DATABASE {db_name};")
    cur.close()
    conn.close()

# Create the database
create_database('angeliteforecast', 'postgres', 'thesis')

DuplicateDatabase: database "angeliteforecast" already exists


In [1]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

def connect_to_database(username, password, host='localhost', port=5432, db_name='angeliteforecast'):
    """Create a database connection."""
    connection_string = f'postgresql://{username}:{password}@{host}:{port}/{db_name}'
    engine = create_engine(connection_string)
    return engine
engine = connect_to_database("postgres", "thesis")



In [ ]:
import pandas as pd


from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

similar_majors_dict = {
    # CCJEF
    "Criminology": ["BS Criminology"], # Combine Criminology and BS Criminology

    
    # SAS
    "ABComm": ["BA Comm"],
    "CommArts": ["BA Comm"],
    "AdvPublicRel": ["BA Comm"], # Area of Specialization
    # "CommArts": "BA Comm",     # Lack of Data | Drop


    # SBA
    "Accounting": ["Accountancy"],
    "Accounting Tech": ["BSAcctgInfoSys"],
    "BSBA-BM-Marketi": ["BusMgt", "BSBA-MktgMgmt"],
    "BussMgmt-HRM": ["BusMgt", "BSBA-HRMgmt"],


    # SEA
    "ECE": ["ELECENG"],
    # "BSCE-CEM": ["CE", "BSCE-CEM"],
    # "BSCE-SE": ["CE", "BSCE-SE"],
    # "BSCE-TE": ["CE", "BSCE-TE"],




    # SED
    "BEED-MajSPED": ["BSNEd"],           # Name change?
    "BEED-SpecialEdu": ["BSNEd"],        # Name change?
    "BPE-SPE": ["BPEd"],                   # Name change?
    "BSED-ValEd": ["BSED-RelValEd"],
    "BSMath": ["Math"],
    

    # SHTM
    "BS EventMgmt": ["BSTM-Events"],


    # SOC
    "BSCSSysDev": ["BSCompsci"],              # New Curriculum
    "BSITAnimation": ["BSEMC-DA"],            # New Curriculum
    "BSITAreaAnimati": ["BSEMC-DA"],          # New Curriculum
    "BSCyberplusPSM": ["BSCybersecurity"],    # Same Major but with Professional Science Master's Degree
    # "BSInfoTech": "BSIT",                 # General IT Major | Lack of Data | Drop
    # "BSITMultiTec": "BSIT",               # Discontinued | Lack of Data | Drop
    "BSITAreaNetAdmi": ["BSITNetAdmin"], 
    "BSITAreaWebDev": ["BSITWebdev"],

    }

# Dictionary for major with incorrect department
incorrect_department_dict = {
    "BSBA-HRM": "SBA",
    # "MAPEH-BSED": "SED",
}

department_dict = {
    "CHTM" : "SHTM",
    "CICT" : "SOC",
}

# List of majors to drop
drop_majors = [
    # MA
    # "MAPEH-BSED",     # Lack of data | Discontinued | SED Department?
    "MBM",              # Lack of data | only 1 student
    "MSEngMgmt",        # Lack of data | only 1 student
    

    # SAS
    # "CommArts",         # Lack of data | only 1 student
    "LanguageLit",      # Lack of data | only 1 student

    # SED
    
    # SHTM
    "BSBATourism",
    "BSTM-Tourism",     # Lack of data | BSTourism? | only 2 students


    # SOC
    "AssCompSci",       # Lack of data | only 1 student
    "BSITMultiTec",     # Lack of data | Discontinued
    "BSInfoTech",       # Lack of data | only 1 student

    # SEA
    "ECETech.",        # Lack of data | Discontinued


    # Extras | Aggregate Counts
    "TOTAL",
    "GRAND TOTAL",
]



In [ ]:
def clean_data(df):
    

    # Step 2: Correct departments

    for major, department in incorrect_department_dict.items():
        df.loc[df["Major"]== major, 'Department'] = department
        
    # Step 3: Rename departments
    df['Department'] = df['Department'].replace(department_dict)

    # Step 4: Drop unwanted majors
    # df = df[~df['Major'].isin(drop_majors)]

    # df = df[~df['Department'].isin(['GS', 'JHS', 'HAUSPELL', 'HAU', 'MA'])]
    df = df.reset_index(drop=True)
    df =df.sort_values(by=["Start_Year", "Semester", "Department", "Major"])
    return df


In [ ]:
df = pd.read_csv("data/Enrollment_Data.csv").sort_values(by=["Start_Year", "Semester", "Department", "Major"])
cpi_df = pd.read_csv("data/CPI_Education.csv").sort_values(by=["Year", "Month"])
inflation_df = pd.read_csv("data/Inflation_Rate.csv").sort_values(by="Start_Year")
admission_df = pd.read_csv("data/Admission_Data.csv").sort_values(by=["Start_Year", "Department"])
hfce_df = pd.read_csv("data/HFCE.csv").sort_values(by=["Start_Year", "Quarter"])

df = clean_data(df)

# Upload data to database
df.to_sql('enrollment', engine, if_exists='replace', index=False)
cpi_df.to_sql('cpi_education', engine, if_exists='replace', index=False)
inflation_df.to_sql('inflation_rate', engine, if_exists='replace', index=False)
admission_df.to_sql('admission', engine, if_exists='replace', index=False)
hfce_df.to_sql('hfce', engine, if_exists='replace', index=False)


54